In [1]:
from collections import Counter
import os
import PyPDF2
from datetime import datetime

wordPerFile = {}

In [2]:
def extract_ticker_and_year(filename):
    # Split the filename into parts
    parts = filename.split('_')
    if len(parts) == 3 and parts[1] == "Annual Report":
        ticker = parts[0]
        year = parts[2]
    else:
        ticker = filename
        year = "-"
    return ticker, year

In [9]:
def count(paths, keywords, output_folder="output"):
    """
    Reads each PDF from the list of paths. If keywords are not found in the text,
    it writes to the output files first before proceeding to the next file.

    :param paths: List of file paths to PDF files.
    :param keywords: List of keywords to look for in the text.
    :param output_folder: Directory to save the output files.
    :return: Word count summary for each file.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    now = datetime.now()
    dateStr = now.strftime("%Y-%m-%d %H-%M-%S")
    sentencesFileName = os.path.join(output_folder, f"sentences-{dateStr}.csv")
    summaryFileName = os.path.join(output_folder, f"summary-{dateStr}.csv")

    # Write headers to the output files
    with open(sentencesFileName, "w", encoding='utf-8', errors='replace') as sentencesFile:
        sentencesFile.write("Ticker,Year,Sentence\n")
    with open(summaryFileName, "w", encoding='utf-8', errors='replace') as summaryFile:
        summaryFile.write(f"Ticker,Year,{'&'.join(keywords)}\n")

    for path in paths:
        print(f"Processing {path}")
        text = ""
        try:
            with open(path, "rb") as f:
                reader = PyPDF2.PdfReader(f)
                for page in reader.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + " "
        except Exception as e:
            text = ""
            print(f"Error processing {path}: {e}")

        filename = os.path.basename(path)
        ticker, year = extract_ticker_and_year(filename.split(".")[0])

        # Check if any keyword exists in the full text
        if not any(keyword.lower() in text.lower() for keyword in keywords):
            # Write to files and proceed to the next file
            print(f"No keywords found in {path}. Writing empty entries...")
            with open(sentencesFileName, "a", encoding='utf-8', errors='replace') as sentencesFile:
                sentencesFile.write(f"{ticker},{year},\n")
            with open(summaryFileName, "a", encoding='utf-8', errors='replace') as summaryFile:
                summaryFile.write(f"{ticker},{year},0\n")
            wordPerFile[f"{ticker} {year}"] = 0
            continue

        # If keywords are found, process the sentences
        sentences = []
        for sentence in text.replace("\n", " ").split('.'):
            for keyword in keywords:
                if keyword.lower() in sentence.lower():
                    sentences.append(sentence.strip())
                    break

        # Save sentences to file
        with open(sentencesFileName, "a", encoding='utf-8', errors='replace') as sentencesFile:
            if not sentences:
                sentencesFile.write(f"{ticker},{year},\n")
            else:
                for sentence in sentences:
                    sentencesFile.write(f"{ticker},{year},{sentence}\n")

        # Save summary to file
        with open(summaryFileName, "a", encoding='utf-8', errors='replace') as summaryFile:
            summaryFile.write(f"{ticker},{year},{len(sentences)}\n")

        wordPerFile[f"{ticker} {year}"] = len(sentences)

    print(f"Output written to:\n- {sentencesFileName}\n- {summaryFileName}")
    return wordPerFile


In [6]:
# Folder containing the files
folder_path = "D:\Hp\OneDrive\shared\\annual report renamed\\2023"

# List to store PDF file paths
file_paths = []

# Iterate through all files in the folder
for filename in os.listdir(folder_path):
    # Check if the file is a PDF
    if filename.endswith(".pdf"):
        full_path = os.path.join(folder_path, filename)
        file_paths.append(full_path)
        print(f"Added: {full_path}")  # Print the file path

# Print the final list of files
print("\nList of PDF files included:")
for path in file_paths:
    print(path)


Added: D:\Hp\OneDrive\shared\annual report renamed\2023\AALI_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ABBA_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ABMM_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ACES_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ACSET_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ACST_Annual_Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADCP_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADES_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADHI_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADMF_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADMG_Annual Report_2023.pdf
Added: D:\Hp\OneDrive\shared\annual report renamed\2023\ADMR_Annual Report_

In [7]:
# Example usage:
# file_paths = ["example1.pdf", "example2.pdf"]  # Replace with your PDF file paths
keywords = ["antitrust violation"]  # Replace with your keywords

In [10]:
# Call the count function directly
result = count(file_paths, keywords)
print("Word Count Summary:", result)

Processing D:\Hp\OneDrive\shared\annual report renamed\2023\AALI_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\AALI_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ABBA_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ABBA_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ABMM_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ABMM_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ACES_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ACES_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ACSET_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report rename

unknown widths : 
[0, IndirectObject(1081, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1084, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1087, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1090, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1093, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1096, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1099, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1102, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1105, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1108, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1111, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1114, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1117, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1120, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1123, 0, 2271619121360)]
unknown widths : 
[0, IndirectObject(1126, 0, 2271619121360)]
unknown 

No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ASLI_Annual report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ASMI_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ASMI_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ASRI_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ASRI_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ASRM_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ASRM_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ASSA_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ASSA_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\sh

c:\Users\Hp\AppData\Local\Programs\Python\Python38\lib\site-packages\PyPDF2\_cmap.py:142: PdfReadWarning: Advanced encoding /90ms-RKSJ-H not implemented yet
  warnings.warn(


No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\BSML_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\BSMR_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\BSMR_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\BSSR_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\BSSR_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\BSWD_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\BSWD_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\BTEK_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\BTEK_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\sh

...

No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\TOYS_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\TPIA_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\TPIA_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\TPMA_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\TPMA_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\TRGU_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\TRGU_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\TRIM_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\TRIM_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\sh

unknown widths : 
[0, IndirectObject(161, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(166, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(171, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(176, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(181, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(186, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(191, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(196, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(201, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(206, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(211, 0, 2271547249040)]


No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\UCID_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\UDNG_Annual Report_2023.pdf


unknown widths : 
[0, IndirectObject(236, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(241, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(246, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(251, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(256, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(261, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(266, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(271, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(276, 0, 2271547249040)]
unknown widths : 
[0, IndirectObject(281, 0, 2271547249040)]


No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\UDNG_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\UFOE_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\UFOE_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\ULTJ_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\ULTJ_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\UNIC_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\UNIC_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\shared\annual report renamed\2023\UNIQ_Annual Report_2023.pdf
No keywords found in D:\Hp\OneDrive\shared\annual report renamed\2023\UNIQ_Annual Report_2023.pdf. Writing empty entries...
Processing D:\Hp\OneDrive\sh